# Introducción a la recuperación de información
Autor: Eric S. Tellez <eric.tellez@infotec.mx><br/>

La recuperación de información se crea a partir de la necesidad de simplificar el acceso y revisión de documentos en grandes colecciones. Estas colecciones pueden ser homogeneas o heterogeneas, tanto en su contenido como en su formato. En su inicio, se consideraban colecciones unicamente de texto pero las necesidades de información se han diversificado, y ahora es común encontrar sistemas de recuperación de información sobre otros datos como imagenes o videos, o include multimodales, esto es que puedan usar diferentes tipos de objetos. Un objeto puede ser un documento de texto, una imagen, o cualquier otro tipo de datos que se desee tener acceso. 

Mantener un sistema de información homogeneo puede simplificar su mantenimiento enormemente, por lo que si es posible, se puede intentar mantener cierta homegeneidad. Para sistemas de fuentes abiertas como puede ser la web, esto será posible.

En general, se puede ver un sistema de recuperación de información en tres grandes partes:

1. Recolección de datos, normalización y modelado matemático.
2. Indexamiento, interpretación de consultas y optimización
3. Agregación y filtrado de resultados, presentación de los mismos


En este curso se visitarán parcialmente todas estas partes. En general se usaran conjuntos de datos previamente recolecados, aunque se invita a usar sus propias colecciones de tal forma que puedan sacar provecho de los temas de manera más amplia. La normalización puede ir desde el simple preprocesamiento de los datos hasta manipulaciones y transformaciones dependientes del dominio. Una vez aplicado el modelado adecuado a las colecciones, las representaciones matemáticas suelen ser vectores de alta dimensión para cada objeto.

En cuanto al indexamiento se utilizarán dos tipos de algoritmos, búsqueda mediante índices invertidos y búsqueda por índices métricos. Ambas tienen sus nichos de aplicación y usarlas adecuadamente requiere conocer los problemas y sus modelados.

La presentación de los resultados puede ser tan simple como una lista de resultados más relevantes y una pequeña muestra del objeto, o más complejo que requiera alguna técnica de visualización. Todo esto dependerá del dominio de aplicación y la naturaleza del sistema de recuperación de información.


# Ejemplos de sistemas de recuperación información

- Google,scholar google, google images, google news, youtube...
- Bing!
- Yahoo search
- Yandex
- DuckDuckGo
- ...

También es común que el sistema de recuperación de información no sea el principal producto, si no más bien un complemento imperdible
como es el caso de los sistemas de streaming de videos, películas, música, redes sociales, etc.

# Ejemplos de implementación de search engines open source
- [Apache Lucene](https://lucene.apache.org/)
- [Apache SOLR](https://solr.apache.org/)
- [ElasticSearch](https://www.elastic.co/es/what-is/elasticsearch)
- [hnswlib](https://github.com/nmslib/hnswlib)
- [FAISS](https://github.com/facebookresearch/faiss)
- [TextSearch.jl](https://github.com/sadit/TextSearch.jl)
- [InvertedFiles.jl](https://github.com/sadit/InvertedFiles.jl)
- [SimilaritySearch.jl](https://github.com/sadit/SimilaritySearch.jl)


# Ejemplo


Construcción de un índice de búsqueda para una colección de mensajes de Twitter. Los mensajes se pueden ver como documentos cortos. Se preprocesan y se modalen en vectores mediante un vocabulario. Después se construye el índice con dichos vectores y se hace posible la búsqueda eficiente. Note que existen etiquetas que se pueden usar para otras tareas; de manera más genérica, se pueden ver como metadatos asociados a los elementos del corpus.

Nota: La primera vez que se corra este notebook los paquetes necesarios se instalaran de manera automática, es posible que tome un tiempo considerable.



In [1]:
using Pkg
Pkg.activate(".")

using TextSearch, InvertedFiles, SimilaritySearch, TextSearch, LinearAlgebra, HypertextLiteral

  Activating project at `~/IR-2022/Unidades`


## Funciones para leer y modelar el corpus

In [2]:
function text_model_and_vectors(corpus)
    textconfig = TextConfig(group_usr=true, group_url=true, del_diac=true, lc=true, group_num=true, nlist=[1], qlist=[])
    model = VectorModel(IdfWeighting(), TfWeighting(), textconfig, corpus)    
    vectors = vectorize_corpus(model, textconfig, corpus)

    (; textconfig, model, vectors)
end


text_model_and_vectors (generic function with 1 method)

In [3]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, _ = read_emojispace()
T = text_model_and_vectors(D.corpus)

@show unique(D.labels), T.model

<h1>Cargando el corpus</h1>

(unique(D.labels), T.model) = (["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡", "🤓", "🙂", "❤", "😠", "😳", "😭", "👌", "🌚", "😅", "😐", "😋", "😕", "😉", "😢", "😱", "💕", "🤣", "😪", "😎", "😏", "😻", "🤭", "💜", "🙈", "🙏", "😥", "😀", "😤", "😩", "😘", "😊", "🙄", "🤗", "😜", "😑", "✨", "😰", "😂", "🎶", "🙃", "🙊", "🤤", "👏", "😁", "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel global_weighting=IdfWeighting(), local_weighting=TfWeighting(), train-voc=42613, train-n=45000, maxoccs=84698})


(["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡"  …  "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel global_weighting=IdfWeighting(), local_weighting=TfWeighting(), train-voc=42613, train-n=45000, maxoccs=84698})

### Note que las etiquetas son emojis, que hay 45k ejemplos y un vocabulario de más de 42k tokens. A continuación se construirá el índice invertido usando el paquete <https://github.com/sadit/TextSearch.jl>, claramente, el código solo funcionará con dicha implementación, pero el procedimiento general aplicaría a otras implementaciones.

In [4]:
invfile = WeightedInvertedFile(length(T.model.voc))
append!(invfile, VectorDatabase(T.vectors))

{WeightedInvertedFile vocsize=42613, n=45000}

In [5]:
function search_and_display(invfile, q, k, D, T)
    res = KnnResult(k)
    @time search(invfile, vectorize(T.model, T.textconfig, q), res)
    L = []
    for (i, (id, dist)) in enumerate(res)
        dist = round(dist, digits=2)
        push!(L, @htl """<tr><td>$i</td> <td>$id => $dist</td> <td>$(D.labels[id])</td> <td>$(D.corpus[id])</td> </tr>""")
       # println(i, " ", id => round(dist, digits=2), " ", D.labels[i], " ", D.corpus[id])
    end

    display(@htl """
    <h3>Resultados para la consulta "$q"</h3>
    <table>
    $L
    </table>
    """)
end

search_and_display (generic function with 1 method)

In [6]:
search_and_display(invfile, "no importa distancia", 5, D, T)
search_and_display(invfile, "cuando nos vemos?", 5, D, T)
search_and_display(invfile, "feliz cumpleaños?", 3, D, T)
search_and_display(invfile, "ola k ase", 3, D, T)
search_and_display(invfile, "mi humilde opinión", 10, D, T)

  0.000307 seconds (18 allocations: 3.344 KiB)


1,35343 => 0.03,💕,No importa la distancia _emo _usr _url
2,24815 => 0.38,😓,Me caga la distancia _emo
3,37163 => 0.52,😃,No importa cuando... _emo _emo _url
4,36425 => 0.57,😑,_usr A ti no te importa _emo
5,1288 => 0.57,😥,"tiempo y distancia, quizás sea eso... _emo"


  0.000284 seconds (19 allocations: 3.359 KiB)


1,30054 => 0.22,😂,Nos vemos!! _emo _emo _emo _emo _url
2,21749 => 0.3,😜,_usr En la tarde nos vemos _emo _emo _emo
3,9076 => 0.32,😻,Nos vemos pronto _usr _usr _emo _emo _url
4,24169 => 0.43,😍,_usr Pronto nos vemos hermana _emo _emo
5,37158 => 0.45,😕,Apenas van _num días que no nos vemos _emo


  0.000144 seconds (18 allocations: 3.328 KiB)


1,8772 => 0.09,🙊,Feliz Cumpleaños _usr _emo _emo _emo _emo _emo
2,26556 => 0.09,❤,_usr Feliz Cumpleaños _emo
3,40229 => 0.09,😁,Feliz Cumpleaños _emo _emo _usr


  0.000036 seconds (17 allocations: 3.172 KiB)


1,43663 => 0.65,😎,"_usr _usr Siempre, todo y todos los días de la semana desde ase muchoooooossss años _emo _emo"
2,23585 => 0.7,😤,_usr k bueno k tu mizmo lo reconozcAszzzz _emo
3,38425 => 0.71,😃,"Despertares!! (By OLA) # _emo _emo en Distrito Federal, Mexico _url"


  0.000171 seconds (17 allocations: 3.266 KiB)


1,2148 => 0.32,🙏,"Bueno, cada quien su vida, ya se, solo es una humilde opinión !! _emo _emo _emo"
2,35818 => 0.51,😍,_usr es la persona más humilde que conozco _emo
3,32858 => 0.62,🤔,_usr Bajo tu opinión fue o no falta? _emo
4,28393 => 0.62,😊,Gracias por compartir la información y tu opinión _emo _url
5,31667 => 0.64,😉,Cuantos Likes para mi humilde oficina _usr _emo _emo @ CocoBongo _url
6,5532 => 0.66,🤗,"Para ser feliz, no escuches la opinión de los demás _emo"
7,12713 => 0.67,👏,_usr Totalmente de acuerdo contigo en todo. Comparto tu opinión _emo _emo _emo _emo
8,990 => 0.72,😌,"Tu opinión no me interesa más. No tengo nada que demostrarte, y si piensas lo peor de mi, MEJOR! _emo"
9,39601 => 0.73,😑,"Lo siento pero mi tez humilde no me deja ser guapo y estar bueno, eso sin mencionar la rodilla que me chingue! _emo _emo"
10,11589 => 0.74,😃,Interesante opinión y espero que sigas escribiendo _usr en mejores espacios !suerte!! _emo _emo _emo _url


### También es posible consultar elementos del mismo corpus, lo cual permité inspeccionar la estructura interna de la colección. Esto será de utilidad más adelante en el curso.

In [7]:
for i in 1:10
    qID = rand(1:length(D.corpus))
    search_and_display(invfile, D.corpus[qID], 7, D, T)
end

  0.001753 seconds (31 allocations: 4.141 KiB)


1,16596 => -0.0,👌,Soy del _num de marzo mano _usr _emo _emo _emo
2,22875 => 0.42,😑,_usr Ya es Marzo... _emo
3,22733 => 0.5,🤭,_usr Soy del _num _emo
4,15065 => 0.56,♡,CDMX te veo en marzo _emo _emo
5,24908 => 0.62,😏,Los de marzo somos los mas guapos _emo
6,30369 => 0.66,🙊,Simplemente el primer proyecto en marzo. _emo _emo _emo
7,20233 => 0.67,🙏,Ya casi veo a mi Chelo _usr _num de marzo llega ya _emo _emo _url


  0.002624 seconds (33 allocations: 4.266 KiB)


1,39876 => 0.0,🤓,A dormir que mañana ese examen no se hace solo _emo
2,44622 => 0.48,😋,A dormir q mañana tenemos examen y a trabajar _emo _emo _emo
3,7793 => 0.55,😥,Un examen más _emo
4,8847 => 0.58,😪,Sólo quiero dormir _emo
5,13762 => 0.58,😫,Sólo quiero dormir _emo
6,21445 => 0.58,😴,Solo quiero dormir _emo
7,17882 => 0.58,🙃,Mañana es mi primer examen final _emo


  0.004151 seconds (50 allocations: 6.688 KiB)


...

1,33319 => -0.0,🤔,"_usr El Respeto en todos los órdenes de la Sociedad y de la vida es Fundamental, cuando Este se pierde, todo Orden o Interacción Fracasa... _emo"
2,20647 => 0.75,💖,_usr A tus órdenes!! _emo _emo
3,26358 => 0.75,🤭,- Con todo respeto _emo
4,31912 => 0.78,👏,_usr _emo Respeto a quien respeto merece _emo _emo _url
5,32591 => 0.78,😢,Pierde los rayados _emo _url
6,43998 => 0.78,😘,A la orden bb _emo _emo _emo _emo _url
7,19016 => 0.79,😁,_usr _usr _usr _usr _usr _usr _usr _usr _usr _usr _usr A la orden _emo


  0.006228 seconds (68 allocations: 6.938 KiB)


1,17360 => -0.0,🤔,"Normalmente cuando veo un tuit de odio pienso en seguida que tienen ""enemigas"" ficticias. O sea tipo fingen tener una vida interesante _emo?"
2,35019 => 0.75,😎,Que interesante _emo
3,11930 => 0.75,😌,Interesante _emo
4,42964 => 0.78,👏,_usr Interesante!! _emo _emo _emo _emo
5,18754 => 0.78,😃,Muy interesante _emo _emo _emo
6,28395 => 0.78,😊,"""TODO"" nunca es mucho cuando se tienen GANAS _emo _emo _emo _emo _emo _emo"
7,1917 => 0.79,🙃,"Odio que un niño me diga ""Me gustas mucho"" cuando yo no siento ni un ""te quiero"" por el _emo"


  0.001535 seconds (31 allocations: 4.172 KiB)


1,13678 => -0.0,😏,_usr Uno de los de Jet Jaguar parecía Leónidas _emo
2,37058 => 0.74,😱,-fue tu novia verdad? parecía? _emo
3,36980 => 0.77,🙄,"Ya era hora, eso ya parecía subasta! _emo _url"
4,8422 => 0.81,😅,"Ir al súper parecía divertido a los _num años, cuando solo decidías que cereal llevar _emo _emo"
5,8892 => 0.81,💔,"Acabo de ver a un niño que se parecía mucho a mi ferru, pero no era él... _emo #HistoriasTristes"
6,33555 => 0.82,😜,#MTVHitsLA luces tan jet Black heart con ese tatuaje tan sexy en tu brazo _usr _emo
7,5224 => 0.83,😓,"Mi foto de perfil de WhatsApp era una de mi perrito, pero la quité porque me dijeron que parecía la señora de los gatos. Soy muy influenciable _emo"


# Actividades

- Revisión de los servicios y empresas mencionados 
- Revisión de los proyectos open source mencionados
- Cargue este notebook y modifique las consultas (necesita Jupyter y el kernel de Julia <https://github.com/JuliaLang/IJulia.jl>)
- Revise la bibliografía sobre recuperación de información, en especial [^SMR2008] se encuentra libre desde <https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf>

# Bibliografía
- [SMR2008] Schütze, H., Manning, C. D., & Raghavan, P. (2008). Introduction to information retrieval (Vol. 39, pp. 234-65). Cambridge: Cambridge University Press.
- [BYN1999] Baeza-Yates, R., & Ribeiro-Neto, B. (1999). Modern information retrieval (Vol. 463). New York: ACM press.